In [ ]:
# Does not need to be executed if ~/.ipython/profile_default/ipython_config.py
# exists and contains get_config().InteractiveShell.ast_node_interactivity = 'all'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
from pathlib import Path
import os
import csv
from collections import defaultdict

Downloaded from https://www.ssa.gov/OACT/babynames/limits.html, the __names__ directory contains, besides __NationalReadMe.pdf__, files whose names are of the form __yob__*xxxx*__.txt__ with "yob" standing for "year of birth" and *xxxx* ranging from 1880 to 2017. These are csv files, with "csv" standing for "comma separated values": each line consists of 3 fields: a first name, __F__ or __M__ for female or male, respectively, and a strictly positive integer for the count of newborns who have been given that name in the year whose value is embedded in the file name. All female names are listed before all male names. For a given gender, data are listed in decreasing order of count. For a given gender and count, names are listed in alphabetical order. For instance, for the oldest year, here are the first 10 lines:

In [ ]:
!head names/yob1880.txt

And here are the last 10 lines:

In [ ]:
!tail names/yob1880.txt

Our first task is to reorganise the data: create a directory __names_per_gender__, create two subdirectories, __female__ and __male__, of __names_per_gender__, and in each of both subdirectories and for each __.txt__ file $F$ in __names__, create a copy of $F$ such that:
* the copy of $F$ in the __female__ subdirectory will consist of the lines for all female names in $F$ with only 2 fields, namely, first name and count, so without __F__, the second field;
* the copy of $F$ in the __male__ subdirectory will consist of the lines for all male names in $F$ with only 2 fields, namely, first name and count, so without __M__, the second field.

To work with directories and files in a platform independent manner, the __Path__ class from the __pathlib__ module is appropriate. One can create __Path__ objects from directory and file names and check whether they exist with __Path__'s __exists()__ method. Given a __Path__ object $P$ for a directory $D$, __Path__ objects for subdirectories of $D$ or for files in $D$ can be created with the __/__ operator, with as first and second operands, $P$ and the subdirectory or file name, respectively; __/__ will produce path names with a separator that is appropriate for the operating system on which code is executed:

In [ ]:
Path('names'), Path('names').exists()
Path('names') / 'yob1880.txt', (Path('names') / 'yob1880.txt').exists()
Path('nonexisting'), Path('nonexisting').exists()
Path('names') / 'yob1800.txt', (Path('names') / 'yob1800.txt').exists()

We first create a __Path__ object for the existing __names__ directory, for the to be created __names_per_gender__ directory, and for the to be created __female__ and __male__ subdirectories of __names_per_gender__:

In [ ]:
names_dirname = Path('names')
names_per_gender_dirname = Path('names_per_gender')
female_subdirname = names_per_gender_dirname / 'female'
male_subdirname = names_per_gender_dirname / 'male'

The __exists()__ function from the __path__ module of the __os__ module also allows one to check whether a directory or file exists. The __removedirs()__ function from the __os__ module allows one to remove a directory. The __mkdir()__ function from the __os__ module allows one to create (make) a directory. Assuming that if the __names_per_gender__ directory exists, then it contains __female__ and __male__ and no other subdirectories, and both __female__ and __male__ are empty directories, then the following code fragment will successfully:

* remove the __female__ directory
* remove the __male__ directory
* remove the then empty __names_per_gender__ directory.

This is necessary for the __os.mkdir(names_per_gender_dirname)__ to then execute successfully, as __os.mkdir()__ raises a __FileExistsError__ error in case it is requested to create a directory that exists already:

In [ ]:
if os.path.exists(names_per_gender_dirname):
    os.removedirs(female_subdirname)
    os.removedirs(male_subdirname)
os.mkdir(names_per_gender_dirname)
os.mkdir(female_subdirname)
os.mkdir(male_subdirname)

We need to process all files in __names__ except for __NationalReadMe.pdf__. We could use the __listdir()__ function from the __os__ module to list all files in __names__ and ignore files not ending in __.txt__:

In [ ]:
for file in os.listdir(names_dirname):
    if not file.endswith('.txt'):
        print(file)

Thanks to the __glob()__ method of the __Path__ class, we can instead generate only the file names of interest. This method uses Unix syntax to create patterns and match file and directory names:

* \* to match a (possibly empty) sequence of characters
* ? to mach a single character
* square brackets to enclose the characters to match.

The following statements illustrate:

In [ ]:
list(names_dirname.glob('*17*'))
list(names_dirname.glob('*2??7*'))
list(names_dirname.glob('*2??[357]*'))

To extract the values  of a csv file, one can of course open the file, read it line by line, and split each line using the comma as separator, but it is cleaner and more robust to instead, let the object returned by __open()__ be the argument of the __reader()__ function of the __csv__ module; that function returns an iterator to generate for each line in the file, the tuple of values on that line. The following code fragment illustrates, printing out all lines in __yob1880.txt__ for counts of female or male name greater than 2000: 

In [ ]:
with open(names_dirname / 'yob1880.txt') as file:
    csv_file = csv.reader(file)
    for name, gender, tally in csv_file:
        if int(tally) > 2_000:
            print(name, gender, tally)

A file such as __yob1880.txt__ is to be processed as one of the files in __names__ whose paths are generated by __glob()__ applied to __names_dirname__. Names and counts extracted from the rows in __yob1880.txt__ are to be written to one of both files with the name __yob1880.txt__ located in the __female__ and __male__ subdirectories of __names_per_gender__. Thanks to the __name__ attribute of a __Path__ object, the paths to both files is conveniently created from the path to __yob1880.txt__ in __names__:

In [ ]:
filename = next(names_dirname.glob('*1880*'))

filename
filename.parent
filename.name
female_subdirname / filename.name
male_subdirname / filename.name

For each __.txt__ file $F$ in __names__, we open, with a single __with__ statement, $F$ for reading purposes, and two files $F_F$ and $F_M$ with the same name as $F$ in the subdirectories __female__ and __male__ of __names_per_gender__, respectively, for writing purposes, with the paths to $F_F$ and $F_M$ created as just described. In parallel to making use of __csv.reader()__, we make use of __csv.writer()__ to write rows of data in a csv file, with successive values properly separated with commas. The dictionary __csv_file_per_gender__ allows one to choose which one of $F_F$ or $F_M$ should be written to. In the last line of the following code fragment, the only purpose of the assignment to **\_** is to suppress Jupyter output: 

In [ ]:
for filename in names_dirname.glob('*.txt'):
    with open(filename) as file,\
         open(female_subdirname / filename.name, 'w') as female_file,\
         open(male_subdirname / filename.name, 'w') as male_file:
        csv_file = csv.reader(file)
        female_csv_file = csv.writer(female_file)
        male_csv_file = csv.writer(male_file)
        csv_file_per_gender = {'F': female_csv_file, 'M': male_csv_file}
        for name, gender, tally in csv_file:
            _ = csv_file_per_gender[gender].writerow((name, tally))

Our second task is to find out the longest intervals of time that separate the years $Y_1$ and $Y_2$ when a name was given (as a male or female name) in both $Y_1$ and $Y_2$, but not in-between. We would like to output the top 10 longest intervals together with the years that start and end the interval, and together with the name that was "forgotten and revived" in that time interval.

To this aim, it is convenient to create a dictionary whose keys are names, with for a given key $N$, the list of years, from oldest to most recent, when $N$ was given once at least. For instance, here are the years when __Franc__ was given as a name: 

In [ ]:
!grep ^Franc, names/*

So __'Franc'__ should be one of the keys, with as value __[1882, 1883, 2001, 2002, 2013]__.

Years will be added one by one to the lists of values as files are processed one by one. Using a simple dictionary, one has to distinguish between creating a key and a value, that should be a list with a single year, and adding a new year to the list that is the value of an existing key: 

In [ ]:
name = 'Franc'
years_per_name = {}
for year in 1882, 1883, 2001, 2002, 2003:
    if not name in years_per_name:
        years_per_name[name] = [year]
        print(f'Processing year {year}: '
              f'creating key "{name}" and value [{year}]'
             )
    else:
        years_per_name[name].append(year)
        print(f'Processing year {year}: '
              f'appending {year} to value for key "{name}"'
             )

years_per_name

A __KeyError__ error is generated when trying to access a nonexisting key:

In [ ]:
name = 'Franc'
years_per_name = {}
years_per_name[name]

When using a __defaultdict__ from the __collections__ module, trying to access a nonexisting key creates the key, together with the default value for the class provided as argument to __defaultdict__:

In [ ]:
name = 'Franc'

years_per_name = defaultdict(int)
print('Creating a key with 0 as default value:')
years_per_name[name];
years_per_name
print('Creating a key with 0 as default value, immediately modified:')
years_per_name = defaultdict(int)
years_per_name[name] += 2; years_per_name

years_per_name = defaultdict(list)
print('Creating a key with [] as default value:')
years_per_name[name]
years_per_name
print('Creating a key with [] as default value, immediately modified:')
years_per_name[name].append(1882); years_per_name

Thanks to default dictionaries, the key __'Franc'__ can be created and years incrementally added to the value list as follows:

In [ ]:
name = 'Franc'
years_per_name = defaultdict(list)
for year in 1882, 1883, 2001, 2002, 2003:
    years_per_name[name].append(year)

years_per_name

Extracting years from filenames is easy:

In [ ]:
int('yob1880.txt'[3: 7])

So creating the full dictionary can be done as follows; we only have to beware that __glob()__ does not return the file names in alphabetical order, so we used __sorted()__ as it is essential that the years that make up the value of a given key of __years_per_name__ are sorted from oldest to most recent:

In [ ]:
years_per_name = defaultdict(list)
for filename in sorted(names_dirname.glob('*.txt')):
    year = int(filename.name[3: 7])
    with open(filename) as file:
        csv_file = csv.reader(file)
        for name, _, _ in csv_file:
            years_per_name[name].append(year)

years_per_name['Franc']

From __years_per_name__, we can create a list of triples of the form $(D, Y, N)$ where $D$ is a year difference, $Y$ is a year that starts a year difference of $D$ (to which $D$ can be added and yield the year that ends the year difference), and $N$ is a name that was given in year $Y$ and only $D$ years later:

In [ ]:
revivals = [[years_per_name[name][i + 1] - years_per_name[name][i],
             years_per_name[name][i],
             name
            ] for name in years_per_name
              for i in range(len(years_per_name[name]) - 1)
           ]

[revivals[i] for i in range(len(revivals))
                 if revivals[i][0] == 2001 - 1883
]

Sorting __revivals__ in reversed order results in a list where:
* year differences are ordered from largest to smallest;
* for a given year difference, years that start the year difference are ordered from most recent to oldest;
* for a given year difference and year that starts the year difference, names are ordered in anti-lexicographic order:

In [ ]:
revivals.sort(reverse = True)
for i in range(10):
    print(f'{revivals[i][2]} was last used in {revivals[i][1]} '
          f'and then again in {revivals[i][1] + revivals[i][0]}, '
          f'{revivals[i][0]} years later.'
         )